In [1]:
f_item = open("track1/item.txt", "r")
n = countlines(f_item)

6095

In [2]:
seekstart(f_item)
item_dict = Dict{Int32, Int32}()
for i = 1:n
    x = split(readline(f_item), '\t')
    item_dict[parse(Int32, x[1])] = i
end

In [3]:
struct User
    id::Int32
    age::Int16
    gender::Int8
    cnt_tweet::Int32
    tag_list::Array{Int32, 1}
end

In [4]:
f_user_profile = open("track1/user_profile.txt", "r")
cnt_user = countlines(f_user_profile)

Tag_dict = Dict{Int32, Int32}()
id_tag = 0
seekstart(f_user_profile)
U = Array{User, 1}()
for i = 1:cnt_user
    x = split(readline(f_user_profile), '\t')
    tag_list = map(t->parse(Int32, t), split(x[5], ';'))
    (tag_list[1] == 0) && (tag_list = Array{Int32, 1}())
    for id in tag_list
        (haskey(Tag_dict, id)) && continue
        id_tag += 1
        Tag_dict[id] = id_tag
    end
    try
        x = map(t->parse(Int32, t), x[1:4])
        x[2] = (x[2] < 1950) ? 0 :
               (1950 ≤ x[2] < 2004) ? ceil((x[2] - 1950) / 2) + 1 : 
               (x[2] ≥ 2004) ? 28 : 29
    catch ArgumentError
        x[2] = "29"
        x = map(t->parse(Int32, t), x[1:4])
    end
    x[4] = x[4] != 0 ? floor(log2(x[4])) : 0
    x[2] += 1
    x[3] += 1
    x[4] += 1
    push!(U, User(x[1], x[2], x[3], x[4], tag_list))
end
println(id_tag)

130304


In [5]:
# build user_dict
user_dict = Dict{Int32, Int32}()
id = 0
for u in U
    id += 1
    user_dict[u.id] = id
end

In [6]:
f_user_keyword = open("track1/user_key_word.txt", "r")
m_user_keyword = countlines(f_user_keyword)

2320895

In [7]:
User_key = Dict{Int32, Array{Tuple{Int32, Float32}, 1}}()
seekstart(f_user_keyword)
Key_dict = Dict{Int32, Int32}()
key_id = 0
for i = 1:m_user_keyword
    line = split(readline(f_user_keyword))
    user_id = user_dict[parse(Int32, line[1])]
    keywords = map(t->split(t, ':'), split(line[2], ';'))
    keywords = map((t->(parse(Int32, t[1]), parse(Float32, t[2]))), keywords)
    User_key[user_id] = keywords
    for x in User_key[user_id]
        if !haskey(Key_dict, x[1])
            key_id += 1
            Key_dict[x[1]] = key_id
        end
    end
end
println(key_id)

255142


In [8]:
f_user_sns = open("track1/user_sns.txt", "r")
len_sns = countlines(f_user_sns)

50655143

In [9]:
seekstart(f_user_sns)
S = Array{Array{Int32, 1}, 1}()
for i = 1:cnt_user
    push!(S, Array{Int32, 1}())
end
len_sns = 1000
for i = 1:len_sns
    x = map(t->user_dict[parse(Int32, t)], split(readline(f_user_sns), '\t'))
    push!(S[x[1]], x[2])
end

In [10]:
f_user_action = open("track1/user_action.txt", "r")
len_action = countlines(f_user_action)

10632833

In [11]:
seekstart(f_user_action)
A = Array{Array{Int32, 1}, 1}()
for i = 1:cnt_user
    push!(A, Array{Int32, 1}())
end
len_action = 1000
for i = 1:len_action
    x = map(t->parse(Int32, t), split(readline(f_user_action), '\t'))
    push!(A[user_dict[x[1]]], user_dict[x[2]])
end

In [12]:
fin = open("track1/pair_train.txt", "r")
m = countlines(fin)

11765848

In [13]:
seekstart(fin)
m = 10000
Ω = Array{Any, 1}()
for i = 1:m
    x = map(t->parse(Int32, t), split(readline(fin), ' '))
    x[1] = user_dict[x[1]]
    push!(x, i)
    push!(Ω, x)
end

In [14]:
function sigmoid(x)
    return 1 / (1 + exp(-x))
end

function cal_timeb(k, t)
    bₜ = (t - d⁻) / (d⁺ - d⁻) * bₜ⁻[k] + (d⁺ - t) / (d⁺ - d⁻) * bₜ⁺[k]
    s = (t - d⁻) % 86400
    bₛ = (s - s⁻) / (s⁺ - s⁻) * bₛ⁻[k] + (s⁺ - s) / (s⁺ - s⁻) * bₛ⁺[k]
    hour = s ÷ 3600
    return bₕ[k, hour + 1] + bₜ + bₛ
end

function cal_timeq(k, t)
    zₜ = (t - d⁻) / (d⁺ - d⁻) * zₜ⁻[k, :] + (d⁺ - t) / (d⁺ - d⁻) * zₜ⁺[k, :]
    s = (t - d⁻) % 86400
    zₛ = (s - s⁻) / (s⁺ - s⁻) * zₛ⁻[k, :] + (s⁺ - s) / (s⁺ - s⁻) * zₛ⁺[k, :]
    return zₜ + zₛ
end

function update_timeb(k, t, Δ)
    global bₜ⁻[k] += (t - d⁻) / (d⁺ - d⁻) * Δ
    global bₜ⁺[k] += (d⁺ - t) / (d⁺ - d⁻) * Δ
    s = (t - d⁻) % 86400
    global bₛ⁻[k] += (s - s⁻) / (s⁺ - s⁻) * Δ
    global bₛ⁺[k] += (s⁺ - s) / (s⁺ - s⁻) * Δ
    hour = s ÷ 3600
    global bₕ[k, hour + 1] += Δ
end

function update_timeq(k, t, Δ)
    global zₜ⁻[k, :] += (t - d⁻) / (d⁺ - d⁻) * Δ
    global zₜ⁺[k, :] += (d⁺ - t) / (d⁺ - d⁻) * Δ
    s = (t - d⁻) % 86400
    global zₛ⁻[k, :] += (s - s⁻) / (s⁺ - s⁻) * Δ
    global zₛ⁺[k, :] += (s⁺ - s) / (s⁺ - s⁻) * Δ
end

update_timeq (generic function with 1 method)

In [15]:
function Train(Ω, k)
    lst_error, iter = 0, 0 
    while true
        try
            iter += 1
            error = 0
            for x in Ω
                u, i₁, i₂ = U[x[1]], U[user_dict[x[3]]], U[user_dict[x[5]]]
                x₃, x₅ = item_dict[x[3]], item_dict[x[5]]
                user_key = Dict{Int32, Float32}()
                user_tag = Dict{Int32, Bool}()
                for key in User_key[x[1]]
                    user_key[key[1]] = key[2]
                end
                for tag in u.tag_list
                    user_tag[tag] = 1
                end
                # Calculate pᵤ
                p_tot = p[k, :, x[1]]
                for xx in S[x[1]]
                    p_tot += y[k, :, xx] / (size(S[x[1]], 1)^0.4)
                end
                for xx in A[x[1]]
                    p_tot += y[k, :, xx] / (size(A[x[1]], 1)^0.5)
                end
                p_tot += yₐ[k, :, u.age, u.gender] + yₜ[k, :, u.cnt_tweet]
                for key in User_key[x[1]]
                    p_tot += key[2] * yₖ[k, :, Key_dict[key[1]]]
                end
                for tag in u.tag_list
                    p_tot += yₚ[k, :, Tag_dict[tag]] / (size(u.tag_list, 1)^0.5)
                end
                # Calculate bᵢ
                b = bᵢ[k, x₅] - bᵢ[k, x₃] 
                    + bₐ[k, u.age, x₅] - bₐ[k, u.age, x₃] 
                    + bᵧ[k, u.gender, x₅] - bᵧ[k, u.gender, x₃]
                    + _bₐ[k, i₂.age, x[1]] - _bₐ[k, i₁.age, x[1]]
                    + _bᵧ[k, i₂.gender, x[1]] - _bᵧ[k, i₁.gender, x[1]]
                    + cal_timeb(k, x[4]) - cal_timeb(k, x[2])
                for key in User_key[user_dict[x[5]]]
                    (!haskey(user_key, key)) && continue
                    b += bₖ[k, Key_dict[key]]
                end
                for key in User_key[user_dict[x[3]]]
                    (!haskey(user_key, key)) && continue
                    b -= bₖ[k, Key_dict[key]]
                end
                for tag in i₂.tag_list
                    (!haskey(user_tag, tag)) && continue
                    b += bₚ[k, Tag_dict[tag]]
                end
                for tag in i₁.tag_list
                    (!haskey(user_tag, tag)) && continue
                    b -= bₚ[k, Tag_dict[tag]]
                end
                # Calculate qᵢ
                qᵢ = q[k, :, x₅] - q[k, :, x₃] + cal_timeq(k, x[4]) - cal_timeq(k, x[2])

                # Predict Error
                r = sigmoid(b + qᵢ[:]' * p_tot)
                error += r

                # Update bᵢ
                bᵢ[k, x₅] += α * r * (1 - r)
                bₐ[k, u.age, x₅] += α * r * (1 - r)
                bᵧ[k, u.gender, x₅] += α * r * (1 - r)
                _bₐ[k, i₂.age, x[1]] += α * r * (1 - r) 
                _bᵧ[k, i₂.gender, x[1]] += α * r * (1 - r)  
                update_timeb(k, x[4], α * r * (1 - r))
                bᵢ[k, x₃] -= α * r * (1 - r)
                bₐ[k, u.age, x₃] -= α * r * (1 - r)
                bᵧ[k, u.gender, x₃] -= α * r * (1 - r)
                _bₐ[k, i₁.age, x[1]] -= α * r * (1 - r)
                _bᵧ[k, i₁.gender, x[1]] -= α * r * (1 - r)
                update_timeb(k, x[2], - α * r * (1 - r))
                for key in User_key[user_dict[x[5]]]
                    (!haskey(user_key, key)) && continue
                    bₖ[k, Key_dict[key]] += α * r * (1 - r)
                end
                for key in User_key[user_dict[x[3]]]
                    (!haskey(user_key, key)) && continue
                    bₖ[k, Key_dict[key]] -= α * r * (1 - r)
                end
                # Update pᵤ
                for xx in S[x[1]]
                    y[k, :, xx] += (α * r * (1 - r) / (size(S[x[1]], 1)^0.4)) * qᵢ
                end
                for xx in A[x[1]]
                    y[k, :, xx] += (α * r * (1 - r) / (size(A[x[1]], 1)^0.5)) * qᵢ
                end
                p[k, :, x[1]] += α * r * (1 - r) * qᵢ
                yₐ[k, :, u.age, u.gender] += α * r * (1 - r) * qᵢ
                yₜ[k, :, u.cnt_tweet] += α * r * (1 - r) * qᵢ
                for key in User_key[x[1]]
                    yₖ[k, :, Key_dict[key[1]]] += key[2] * α * r * (1 - r) * qᵢ
                end
                for tag in u.tag_list
                    yₚ[k, :, Tag_dict[tag]] += α * r * (1 - r) / (size(u.tag_list, 1)^0.5) * qᵢ
                end
                # Update qᵢ
                q[k, :, x₅] += α * r * (1 - r) * p_tot
                update_timeq(k, x[4], α * r * (1 - r) * p_tot)
                q[k, :, x₃] -= α * r * (1 - r) * p_tot
                update_timeq(k, x[2], - α * r * (1 - r) * p_tot)
            end
            error /= _m
            (iter % 1 == 0) && println("$iter $error")
            lst_error = error
        catch InterruptException
            break
        end
    end
    println(lst_error)
end

Train (generic function with 1 method)

In [16]:
function Predict(Ω, k)
    error = Array{Any, 1}()
    for x in Ω
        u, i₁, i₂ = U[x[1]], U[user_dict[x[3]]], U[user_dict[x[5]]]
        x₃, x₅ = item_dict[x[3]], item_dict[x[5]]
        user_key = Dict{Int32, Float32}()
        user_tag = Dict{Int32, Bool}()
        for key in User_key[x[1]]
            user_key[key[1]] = key[2]
        end
        for tag in u.tag_list
            user_tag[tag] = 1
        end
        # Calculate pᵤ
        p_tot = p[k, :, x[1]]
        for xx in S[x[1]]
            p_tot += y[k, :, xx] / (size(S[x[1]], 1)^0.4)
        end
        for xx in A[x[1]]
            p_tot += y[k, :, xx] / (size(A[x[1]], 1)^0.5)
        end
        p_tot += yₐ[k, :, u.age, u.gender] + yₜ[k, :, u.cnt_tweet]
        for key in User_key[x[1]]
            p_tot += key[2] * yₖ[k, :, Key_dict[key[1]]]
        end
        for tag in u.tag_list
            p_tot += yₚ[k, :, Tag_dict[tag]] / (size(u.tag_list, 1)^0.5)
        end
        # Calculate bᵢ
        b = bᵢ[k, x₅] - bᵢ[k, x₃] 
            + bₐ[k, u.age, x₅] - bₐ[k, u.age, x₃] 
            + bᵧ[k, u.gender, x₅] - bᵧ[k, u.gender, x₃]
            + _bₐ[k, i₂.age, x[1]] - _bₐ[k, i₁.age, x[1]]
            + _bᵧ[k, i₂.gender, x[1]] - _bᵧ[k, i₁.gender, x[1]]
            + cal_timeb(k, x[4]) - cal_timeb(k, x[2])
        for key in User_key[user_dict[x[5]]]
            (!haskey(user_key, key)) && continue
            b += bₖ[k, Key_dict[key]]
        end
        for key in User_key[user_dict[x[3]]]
            (!haskey(user_key, key)) && continue
            b -= bₖ[k, Key_dict[key]]
        end
        for tag in i₂.tag_list
            (!haskey(user_tag, tag)) && continue
            b += bₚ[k, Tag_dict[tag]]
        end
        for tag in i₁.tag_list
            (!haskey(user_tag, tag)) && continue
            b -= bₚ[k, Tag_dict[tag]]
        end
        # Calculate qᵢ
        qᵢ = q[k, :, x₅] - q[k, :, x₃] + cal_timeq(k, x[4]) - cal_timeq(k, x[2])

        # Predict Error
        r = sigmoid(b + qᵢ[:]' * p_tot)
        push!(error, (r > 0.5))
    end
    return error
end

Predict (generic function with 1 method)

In [ ]:
using StatsBase

K = 5
W = zeros(m, 1)
w = zeros(K, 1)
for i = 1:m
    W[i] = 1/m
end

d = 50
α = 0.001
d⁻, d⁺ = 1318348785, 1322668798
s⁻, s⁺ = 0, 86400
bₕ = rand(K, 24)
bₖ = rand(K, key_id)
bₜ⁻, bₜ⁺ = rand(K, 1), rand(K, 1)
bₛ⁻, bₛ⁺ = rand(K, 1), rand(K, 1)
zₜ⁻, zₜ⁺ = rand(K, d), rand(K, d)
zₛ⁻, zₛ⁺ = rand(K, d), rand(K, d)
p = rand(K, d, cnt_user)
y = rand(K, d, cnt_user)
q = rand(K, d, n)
bᵢ = rand(K, n)
bₐ = rand(K, 30, n)
bᵧ = rand(K, 4, n)
_bₐ = rand(K, 30, cnt_user)
_bᵧ = rand(K, 4, cnt_user)
bₚ = rand(K, id_tag)
yₐ = rand(K, d, 30, 4)
yₜ = rand(K, d, 16)
yₖ = rand(K, d, key_id)
yₚ = rand(K, d, id_tag)

_m = 5000000

for k = 1:K
    Ωₖ = StatsBase.sample(Ω, ProbabilityWeights(W[:]), _m; replace=true, ordered=true)
    println(Ωₖ[1:10])
    Train(Ωₖ, k)
    error = Predict(Ωₖ, k)
    w[k] = 1 - sum(error)/_m
    for i = 1:_m
        (error[i]) && (W[Ωₖ[i][6]] *= w[k] / (1 - w[k]))
    end
    W /= sum(W)
    println(w[k])
    println(W[1:40])
end

In [22]:
function read_from_log_test(f)
    x = split(readline(f), '\t')
    x = map(t->parse(Int32, t), x[1:4])
    return x
end

function test(f_rec_log_test, fout, key)
    m_test = countlines(f_rec_log_test)
    seekstart(f_rec_log_test)
    user_cnt, i = 0, 1
    cur = read_from_log_test(f_rec_log_test)
    #m_test = 200
    while i <= m_test
        j = i + 1
        user_cnt += 1
        _Ω = Array{Any, 1}()
        push!(_Ω, cur)
        while j <= m_test
            cur = read_from_log_test(f_rec_log_test)
            (cur[1] != _Ω[1][1]) && break
            j += 1
            push!(_Ω, cur)
        end
        (i ÷ 5000000 != j ÷ 5000000) && println(i, ' ', j)
        dict = Array{Any, 1}()
        x₁ = user_dict[_Ω[1][1]]
        u = U[x₁]
        user_key = Dict{Int32, Float32}()
        user_tag = Dict{Int32, Bool}()
        for key in User_key[x₁]
            user_key[key[1]] = key[2]
        end
        for tag in u.tag_list
            user_tag[tag] = 1
        end
        p_tot = Array{Any, 1}()
        for k = 1:K
            push!(p_tot, p[k, :, x₁])
            for xx in S[x₁]
                p_tot[k] += y[k, :, xx] / (size(S[x₁], 1)^0.4)
            end
            for xx in A[x₁]
                p_tot[k] += y[k, :, xx] / (size(A[x₁], 1)^0.5)
            end
            p_tot[k] += yₐ[k, :, u.age, u.gender] + yₜ[k, :, u.cnt_tweet]
            for key in User_key[x₁]
                p_tot[k] += key[2] * yₖ[k, :, Key_dict[key[1]]]
            end
            for tag in u.tag_list
                p_tot[k] += yₚ[k, :, Tag_dict[tag]] / (size(u.tag_list, 1)^0.5)
            end
        end
        for x in _Ω[1:j-i]
            x₂ = item_dict[x[2]]
            i₂ = U[user_dict[x[2]]] 
            score = 0
            for k = 1:K
                b = bᵢ[k, x₂] + cal_timeb(k, x[4]) + bₐ[k, u.age, x₂] + bᵧ[k, u.gender, x₂] + _bₐ[k, i₂.age, x₁] + _bᵧ[k, i₂.gender, x₁]
                for key in User_key[user_dict[x[2]]]
                    (!haskey(user_key, key)) && continue
                    b += bₖ[k, Key_dict[key]]
                end
                for tag in i₂.tag_list
                    (!haskey(user_tag, tag)) && continue
                    b += bₚ[k, Tag_dict[tag]]
                end
                qᵢ = q[k, :, x₂] + cal_timeq(k, x[4])
                score += log((1-w[k])/w[k]) * sigmoid(b + qᵢ[:]' * p_tot[k])
            end
            push!(dict, (score, x[2]))
        end
        sort!(dict)
        println(dict)
        write(fout, "$(_Ω[1][1]),")
        for k = 1:min(size(dict, 1), 3)
            t = size(dict, 1) - k + 1
            write(fout, ((k == 1) ? "" : " ") * "$(dict[t][2])")
        end
        write(fout, "\n")
        i = j
    end
end

fout = open("test.csv", "w")
write(fout, "id,clicks\n")

test(open("track1/rec_log_test_pub.txt", "r"), fout, "Public")
test(open("track1/rec_log_test_pri.txt", "r"), fout, "Private")

close(fout)

Any[(2.30986, 713240), (2.30986, 1760654), (2.30986, 592273), (2.30986, 2417022), (2.30986, 2105722), (2.30986, 1774684), (2.30986, 1774687), (2.30986, 651589), (2.30986, 2105511), (2.30986, 1774708), (2.30986, 713242), (2.30986, 1774718), (2.30986, 1870054), (2.30986, 699215), (2.30986, 441554), (2.30986, 1774422), (2.30986, 1760401), (2.30986, 2105550), (2.30986, 1837210), (2.30986, 2274594), (2.30986, 2022076), (2.30986, 655717), (2.30986, 1774463), (2.30986, 461710), (2.30986, 760113), (2.30986, 1760338), (2.30986, 1774455), (2.30986, 1606596), (2.30986, 1774457), (2.30986, 520267), (2.30986, 1494039), (2.30986, 1775024), (2.30986, 2018604), (2.30986, 1760374), (2.30986, 1774461), (2.30986, 1774969), (2.30986, 1562118), (2.30986, 665932), (2.30986, 725380), (2.30986, 535240), (2.30986, 713254), (2.30986, 1606574), (2.30986, 713229), (2.30986, 458026), (2.30986, 704241), (2.30986, 2105482), (2.30986, 1774550), (2.30986, 1774963), (2.30986, 1774452), (2.30986, 2105595), (2.30986, 160